# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal

In [2]:
models = Models.load('../models')   # loading neural networks

INFO:tensorflow:Restoring parameters from ../models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ../models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ../models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ../models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ../models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ../models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ../models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ../models/decl_model/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, True

# you sit West and hold:
hand = '73.KJ83.AT2.T962'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['PAD_START', '1D', '1S']

# what's your bid?

In [4]:
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models)

bot_bid.bid(['PAD_START', '1D', '1S']).bid

'X'

In [5]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(['PAD_START', '1D', '1S', 'X', '2S', '3H', 'PASS'])
bid.bid

'4H'

In [6]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': '4H',
  'insta_score': 0.658929705619812,
  'expected_score': 104.60807762839579},
 {'call': 'PASS',
  'insta_score': 0.3114379048347473,
  'expected_score': 41.22101373108897}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [7]:
bid.samples

['J9x.xxx.QJ8.K8xx Q8x.QTxx.Kxxx.AQ AKTxx.A9.9xx.Jxx xx.KJ8x.ATx.T9xx',
 'Jxxx.xx.Q9.K8xxx K8x.AQ9x.K8xxx.Q AQT9.Txx.Jxx.AJx xx.KJ8x.ATx.T9xx',
 '8xx.T9.KQ8x.Q8xx AJ9.Qxxx.J9xx.Ax KQTxx.Axx.xx.KJx xx.KJ8x.ATx.T9xx',
 'Jxx.xx.J98x.AJxx K98.AQ9x.Qxxx.Kx AQTxx.Txx.Kx.Q8x xx.KJ8x.ATx.T9xx',
 'QT9.Txxx.Kx.8xxx J8x.A9xx.QJ8xx.A AKxxx.Q.9xx.KQJx xx.KJ8x.ATx.T9xx',
 'QJ8x.QT9.9xx.A8x xx.Axxx.KQ8x.KQx AKT9x.xx.Jxx.Jxx xx.KJ8x.ATx.T9xx',
 '8xx.9xx.J98.KQJ8 QTx.QTxx.KQxx.Ax AKJ9x.Ax.xxx.xxx xx.KJ8x.ATx.T9xx',
 'QJ9x.xxxx.8xx.Ax Tx.QT9x.KQJ9xx.K AK8xx.A.x.QJ8xxx xx.KJ8x.ATx.T9xx',
 'A98x.9xx.xx.Jxxx Kx.Qxxx.QJxx.AQx QJTxx.AT.K98x.K8 xx.KJ8x.ATx.T9xx',
 'J8x.9xx.xxxx.KQ8 T9x.AQTx.KQJx.Jx AKQxx.xx.98.Axxx xx.KJ8x.ATx.T9xx']

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [8]:
# we got some samples above. if we want more, we can always get more

sample_hands = bot_bid.sample_hands(['PAD_START', '1D', '1S', 'X', '2S', '3H', 'PASS'])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

QT8.Q9x.98x.Kxxx KJx.ATxx.KJxx.Q8 A9xxx.xx.Qxx.AJx xx.KJ8x.ATx.T9xx
98x.Qx.xxx.A8xxx Ax.xxxx.KQ9xx.KQ KQJTxx.AT9.J8.Jx xx.KJ8x.ATx.T9xx
Txx.Q9x.8xx.Axxx A8x.Axxx.KJ9x.Q8 KQJ9x.Tx.Qxx.KJx xx.KJ8x.ATx.T9xx
Jxx.9x.Kxx.KJxxx AT9.AQxx.QJ9xx.Q KQ8xx.Txx.8x.A8x xx.KJ8x.ATx.T9xx
T8x.Tx.Kxx.Axxxx Qxx.AQxx.QJ98x.Q AKJ9x.9xx.xx.KJ8 xx.KJ8x.ATx.T9xx
9xx.Q9xx.xx.AJxx A8x.ATx.QJ98x.Kx KQJTx.xx.Kxx.Q8x xx.KJ8x.ATx.T9xx
Jxx.9xxx.xx.AKxx QT8.AQxx.QJ9xx.Q AK9xx.T.K8x.J8xx xx.KJ8x.ATx.T9xx
QTxx.Q9xx.98.Qxx Kx.ATxx.KQJxx.Jx AJ98x.x.xxx.AK8x xx.KJ8x.ATx.T9xx
T9xx.Q9x.98.Kxxx K8x.ATxx.Kxxx.Ax AQJx.xx.QJxx.QJ8 xx.KJ8x.ATx.T9xx
KJT.9x.Jxxx.Q8xx A9x.Axxx.KQ9x.xx Q8xxx.QTx.8x.AKJ xx.KJ8x.ATx.T9xx
98x.T9x.Jxx.KQJx QTx.Axxx.K8xx.Ax AKJxx.Qx.Q9x.8xx xx.KJ8x.ATx.T9xx
A8x.xx.98xx.Jxxx Jxx.ATxx.KQJx.K8 KQT9x.Q9x.xx.AQx xx.KJ8x.ATx.T9xx
T9xx.T9x.Qxx.A8x AQ.Qxxx.J98x.KQx KJ8xx.Ax.Kxx.Jxx xx.KJ8x.ATx.T9xx
Jxx.ATx.Q9x.J8xx Ax.9xxx.K8xx.AQx KQT98x.Qx.Jxx.Kx xx.KJ8x.ATx.T9xx
K9x.xxx.8xx.A8xx AJ8.QT9x.KQJx.Jx QTxxx.Ax.9xx.K

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [9]:
hands = random_deal().split()
hands # N, E, S, W

['Q95.AQ42.KJ7.K92',
 'KT643.JT965.T3.7',
 'AJ72.87.8652.T85',
 '8.K3.AQ94.AQJ643']

In [10]:
bidder_bots = [BotBid([False, False], hand, models) for hand in hands]

In [11]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['1N', 'PASS', 'PASS', 'X', 'PASS', 'PASS', 'PASS']